# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [396]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. '''

## 2. Ask for parameters

In [419]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [420]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

*  $i = 1, 2$: Index for the two different raw materials 
*  $j = 1, 2$: Index for the two different drugs

## Parameters:

*  $P_j$:  Profit per unit of drug $j$ sold
*  $C_i$:  Purchasing cost per unit of raw material $i$ 
*  $OC_j$: Operational cost per unit of drug $j$ produced
*  $B$: Total available budget for purchasing raw materials and producing the drugs
*  $Cap_j$: Maximum production capacity for drug $j$ 
*  $S_i$: Maximum storage capacity for raw material $i$
*  $RA_j$: Required amount of active ingredient in drug  $j$
*  $EA_{ij}$: Nominal amount of active ingredient extracted from one unit of raw material $i$ for drug $j$ 
*  $\alpha_1$: Maximum deviation from $EA_{1j}$ (0.5%)
*  $\alpha_2$: Maximum deviation from $EA_{2j}$ (2%) 

## Decision Variables:

*  $x_i$: Amount of raw material $i$ purchased
*  $y_j$: Amount of drug $j$ produced 


# 2. Ask for objective

In [421]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [422]:
Markdown(response2.text)

$$\text{Maximize } \sum_{j=1}^{2} P_j y_j - \sum_{i=1}^{2} C_i x_i - \sum_{j=1}^{2} OC_j y_j $$ 


# 3. Ask for constraints

In [423]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [424]:
Markdown(response3.text)

$$\begin{aligned}
& \text{subject to} \\
&\sum_{i=1}^{2} C_ix_i + \sum_{j=1}^{2}OC_jy_j \leq B \\
& y_j \leq Cap_j  &\forall j = 1,2 \\
& x_i \leq  S_i &\forall i = 1,2\\
&\sum_{i=1}^{2} (EA_{ij} - \alpha_i EA_{ij})x_i \geq RA_j  &\forall j = 1,2\\
& x_i, y_j \geq 0  &\forall i = 1,2; \forall j = 1,2 
\end{aligned}$$ 


## 3. Generate the pyomo code

In [425]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [426]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data 
P = {1: 100, 2: 150}  
C = {1: 20, 2: 30}  
OC = {1: 10, 2: 15}  
B = 10000 
Cap = {1: 500, 2: 400}  
S = {1: 800, 2: 600} 
RA = {1: 100, 2: 120}  
EA = {(1, 1): 0.12, (1, 2): 0.15, (2, 1): 0.10, (2, 2): 0.13}  
alpha = {1: 0.005, 2: 0.02}  

# Model creation
model = pyo.ConcreteModel()

# Set definition
model.i = pyo.Set(initialize=[1, 2]) # Raw materials
model.j = pyo.Set(initialize=[1, 2]) # Drugs 

# Variable definition
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals) # Amount of raw material purchased
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals) # Amount of drug produced

# Objective function
model.profit = pyo.Objective(
    expr=sum(P[j] * model.y[j] for j in model.j) - sum(C[i] * model.x[i] for i in model.i) - sum(OC[j] * model.y[j] for j in model.j),
    sense=pyo.maximize
)

# Constraints
model.budget = pyo.Constraint(expr=sum(C[i] * model.x[i] for i in model.i) + sum(OC[j] * model.y[j] for j in model.j) <= B)

model.capacity = pyo.ConstraintList()
for j in model.j:
    model.capacity.add(model.y[j] <= Cap[j])

model.storage = pyo.ConstraintList()
for i in model.i:
    model.storage.add(model.x[i] <= S[i])

model.ingredient = pyo.ConstraintList()
for j in model.j:
    model.ingredient.add(sum((EA[i, j] - alpha[i] * EA[i, j]) * model.x[i] for i in model.i) >= RA[j])

# Solve the model
solver = pyo.SolverFactory('glpk')  
results = solver.solve(model)

# Print the results
print(results)
print("Profit =", pyo.value(model.profit))
print("x =", {i: pyo.value(model.x[i]) for i in model.i})
print("y =", {j: pyo.value(model.y[j]) for j in model.j})
``` 


## 4. Input problem data and try running the generated code

In [429]:
import pyomo.environ as pyo

# Sample data 
P = {1: 100, 2: 150}  
C = {1: 20, 2: 30}  
OC = {1: 10, 2: 15}  
B = 10000 
Cap = {1: 500, 2: 400}  
S = {1: 800, 2: 600} 
RA = {1: 100, 2: 120}  
EA = {(1, 1): 0.12, (1, 2): 0.15, (2, 1): 0.10, (2, 2): 0.13}  
alpha = {1: 0.005, 2: 0.02}  

# Model creation
model = pyo.ConcreteModel()

# Set definition
model.i = pyo.Set(initialize=[1, 2]) # Raw materials
model.j = pyo.Set(initialize=[1, 2]) # Drugs 

# Variable definition
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals) # Amount of raw material purchased
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals) # Amount of drug produced

# Objective function
model.profit = pyo.Objective(
    expr=sum(P[j] * model.y[j] for j in model.j) - sum(C[i] * model.x[i] for i in model.i) - sum(OC[j] * model.y[j] for j in model.j),
    sense=pyo.maximize
)

# Constraints
model.budget = pyo.Constraint(expr=sum(C[i] * model.x[i] for i in model.i) + sum(OC[j] * model.y[j] for j in model.j) <= B)

model.capacity = pyo.ConstraintList()
for j in model.j:
    model.capacity.add(model.y[j] <= Cap[j])

model.storage = pyo.ConstraintList()
for i in model.i:
    model.storage.add(model.x[i] <= S[i])

model.ingredient = pyo.ConstraintList()
for j in model.j:
    model.ingredient.add(sum((EA[i, j] - alpha[i] * EA[i, j]) * model.x[i] for i in model.i) >= RA[j])

# Solve the model
solver = pyo.SolverFactory('glpk')  
results = solver.solve(model)

# Print the results
print(results)
#print("Profit =", model.profit())
# print("x =", {i: pyo.value(model.x[i]) for i in model.i})
# print("y =", {j: pyo.value(model.y[j]) for j in model.j})


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 4
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0404965877532959



## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [430]:
print(response.text)

## Indices:

*  $i = 1, 2$: Index for the two different raw materials 
*  $j = 1, 2$: Index for the two different drugs

## Parameters:

*  $P_j$:  Profit per unit of drug $j$ sold
*  $C_i$:  Purchasing cost per unit of raw material $i$ 
*  $OC_j$: Operational cost per unit of drug $j$ produced
*  $B$: Total available budget for purchasing raw materials and producing the drugs
*  $Cap_j$: Maximum production capacity for drug $j$ 
*  $S_i$: Maximum storage capacity for raw material $i$
*  $RA_j$: Required amount of active ingredient in drug  $j$
*  $EA_{ij}$: Nominal amount of active ingredient extracted from one unit of raw material $i$ for drug $j$ 
*  $\alpha_1$: Maximum deviation from $EA_{1j}$ (0.5%)
*  $\alpha_2$: Maximum deviation from $EA_{2j}$ (2%) 

## Decision Variables:

*  $x_i$: Amount of raw material $i$ purchased
*  $y_j$: Amount of drug $j$ produced 



In [431]:
print(response2.text)

$$\text{Maximize } \sum_{j=1}^{2} P_j y_j - \sum_{i=1}^{2} C_i x_i - \sum_{j=1}^{2} OC_j y_j $$ 



In [432]:
print(response3.text)

$$\begin{aligned}
& \text{subject to} \\
&\sum_{i=1}^{2} C_ix_i + \sum_{j=1}^{2}OC_jy_j \leq B \\
& y_j \leq Cap_j  &\forall j = 1,2 \\
& x_i \leq  S_i &\forall i = 1,2\\
&\sum_{i=1}^{2} (EA_{ij} - \alpha_i EA_{ij})x_i \geq RA_j  &\forall j = 1,2\\
& x_i, y_j \geq 0  &\forall i = 1,2; \forall j = 1,2 
\end{aligned}$$ 



In [433]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data 
P = {1: 100, 2: 150}  
C = {1: 20, 2: 30}  
OC = {1: 10, 2: 15}  
B = 10000 
Cap = {1: 500, 2: 400}  
S = {1: 800, 2: 600} 
RA = {1: 100, 2: 120}  
EA = {(1, 1): 0.12, (1, 2): 0.15, (2, 1): 0.10, (2, 2): 0.13}  
alpha = {1: 0.005, 2: 0.02}  

# Model creation
model = pyo.ConcreteModel()

# Set definition
model.i = pyo.Set(initialize=[1, 2]) # Raw materials
model.j = pyo.Set(initialize=[1, 2]) # Drugs 

# Variable definition
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals) # Amount of raw material purchased
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals) # Amount of drug produced

# Objective function
model.profit = pyo.Objective(
    expr=sum(P[j] * model.y[j] for j in model.j) - sum(C[i] * model.x[i] for i in model.i) - sum(OC[j] * model.y[j] for j in model.j),
    sense=pyo.maximize
)

# Constraints
model.budget = pyo.Constraint(expr=sum(C[i] * model.x[i] for i in model.i) + sum(OC[j] * model.y[j] for j in model.j) 